In [1]:
import requests
import json
import os

# load the environment variables
from dotenv import load_dotenv
load_dotenv()

# Bring your packages onto the path
import sys
sys.path.append(os.path.abspath(os.path.join('..', 'src')))

# import functions from src/mlb_scores.py
from mlb_scores import get_mlb_scores, get_mlb_odds, get_correct_upsets, create_tweet

In [2]:
# get yesterday's date as string
import datetime
yesterday = datetime.date.today() - datetime.timedelta(days=1)
yesterday = yesterday.strftime('%Y-%m-%d')

# date = '2022-08-14'
date = yesterday

In [3]:
score_match_dict = get_mlb_scores(date)
score_match_dict

Connected to MLB API


{74196: {'home_team': 'Rangers',
  'away_team': 'Astros',
  'home_score': 2,
  'away_score': 4,
  'winner': 'Away'},
 74201: {'home_team': 'White Sox',
  'away_team': 'Royals',
  'home_score': 7,
  'away_score': 9,
  'winner': 'Away'},
 74207: {'home_team': 'Brewers',
  'away_team': 'Pirates',
  'home_score': 2,
  'away_score': 4,
  'winner': 'Away'},
 74199: {'home_team': 'Angels',
  'away_team': 'Yankees',
  'home_score': 4,
  'away_score': 7,
  'winner': 'Away'},
 74203: {'home_team': 'Diamondbacks',
  'away_team': 'Phillies',
  'home_score': 12,
  'away_score': 3,
  'winner': 'Home'},
 74213: {'home_team': 'Giants',
  'away_team': 'Padres',
  'home_score': 3,
  'away_score': 4,
  'winner': 'Away'},
 74212: {'home_team': 'Rangers',
  'away_team': 'Astros',
  'home_score': 3,
  'away_score': 5,
  'winner': 'Away'},
 74208: {'home_team': 'Brewers',
  'away_team': 'Pirates',
  'home_score': 6,
  'away_score': 1,
  'winner': 'Home'},
 74197: {'home_team': 'Giants',
  'away_team': 'Padre

In [4]:
odds_match_dict = get_mlb_odds(score_match_dict, date)
odds_match_dict

{74196: 'Away',
 74201: 'Home',
 74207: 'Home',
 74199: 'Away',
 74213: 'Away',
 74203: 'Away',
 74212: 'Away',
 74208: 'Home',
 74197: 'Away',
 74216: 'Home',
 74220: 'Away',
 74221: 'Away',
 74210: 'Away',
 74211: 'Home',
 74209: 'Home',
 74215: 'Away',
 74218: 'Home',
 74222: 'Home'}

In [5]:
odds_match_dict

{74196: 'Away',
 74201: 'Home',
 74207: 'Home',
 74199: 'Away',
 74213: 'Away',
 74203: 'Away',
 74212: 'Away',
 74208: 'Home',
 74197: 'Away',
 74216: 'Home',
 74220: 'Away',
 74221: 'Away',
 74210: 'Away',
 74211: 'Home',
 74209: 'Home',
 74215: 'Away',
 74218: 'Home',
 74222: 'Home'}

In [6]:
# get lists of correct and incorrect predictions
correct_ids = []
upset_ids = []

for match_id in odds_match_dict.keys():
    if odds_match_dict[match_id]!=score_match_dict[match_id]['winner']:
       upset_ids.append(match_id) 
    else:
        correct_ids.append(match_id)

def create_tweet(correct_ids, upset_ids, score_match_dict, date='2022-08-11'):
    tweet_text = str('#MLB Scores {date}\n\n').format(date=date)
    tweet_text=tweet_text+str("UPSETS: \n")
    for id in upset_ids:
        score_match_dict[id]
        tweet_text=tweet_text+str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score'])

    tweet_text=tweet_text+str("\n EXPECTED: \n")
    for id in correct_ids:
        score_match_dict[id]
        tweet_text=tweet_text+(str("{home_team}: {home_score} - {away_team}: {away_score} \n").format(
            home_team=score_match_dict[id]['home_team'], 
            home_score=score_match_dict[id]['home_score'], 
            away_team=score_match_dict[id]['away_team'], 
            away_score=score_match_dict[id]['away_score']))

    if len(tweet_text)>=280:
        tweet_text_list = tweet_text.split('EXPECTED')
        tweet_text_list[1]='EXPECTED'+tweet_text_list[1]
    else:
        tweet_text_list=[tweet_text]

    return tweet_text_list

tweet_text_list = create_tweet(correct_ids, upset_ids, score_match_dict, date=date)

In [7]:
for tweet_text in (tweet_text_list):
    print(tweet_text)

#MLB Scores 2022-08-31

UPSETS: 
White Sox: 7 - Royals: 9 
Brewers: 2 - Pirates: 4 
Diamondbacks: 12 - Phillies: 3 
Guardians: 0 - Orioles: 4 
Nationals: 5 - Athletics: 1 
Jays: 5 - Cubs: 7 
Twins: 5 - Red Sox: 6 

 
EXPECTED: 
Rangers: 2 - Astros: 4 
Angels: 4 - Yankees: 7 
Giants: 3 - Padres: 4 
Rangers: 3 - Astros: 5 
Brewers: 6 - Pirates: 1 
Giants: 4 - Padres: 5 
Reds: 3 - Cardinals: 5 
Marlins: 1 - Rays: 2 
Mets: 2 - Dodgers: 1 
Tigers: 3 - Mariners: 5 
Braves: 3 - Rockies: 2 

